# Drawing Guesser 
## Pedro Luiz & Gabriel Zezze

### O dataset
O drawing guesser se baseia no [dataset de desenhos dispopnibilizados pelo Google](https://github.com/googlecreativelab/quickdraw-dataset) gerado através do jogo [QuickDraw](https://quickdraw.withgoogle.com/).

O dataset completo consiste de 150 mil desenhos que são imagens bitmap de 28x28 para cada categoria sendo que há 345 categorias disponibilizadas.

O intuito do drawing guesser é simular o proprio QuickDraw, ou seja, fazer classificações do desenho enquanto o jogador desenha, porém com um número limitado de catogrias (x) e uma rede neural própria treinada a partir do dataset disponibilizado.

### O problema

Este é um problema de classifição multiclasse dos desenhos dispoinibilizados no dataset. Para realizar essa classificação vamos utilizar uma rede convulacional que será explicada detalhadamente abaixo.


In [2]:
#IMPORTS
import pandas as pd
import numpy as np
import ujson as json
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, BatchNormalization
import tensorflow as tf
from sklearn.metrics import classification_report

In [3]:
#DECLARE CONSTANTS
IMG_HEIGHT = 28
IMG_WIDTH = 28
NUM_FILES = 35000
RF_NUM_FILES = 10000
N_EPOCHS = 300
RANDOM_SEED = 42

### Extração de dados

O dataset utilizado disponibiliza arquivos do tipo numpy bit array, que quando lidos com a função `numpy.load` retorna um numpy array. Contudo, esse array possui apenas uma dimensão. Dessa forma, é necessário fazer o reshape de cada imagem lida para se tornar uma matriz 28x28. 

In [4]:
#GETTING ALL FILE NAMES
data_files = os.listdir('./data/')
data_files = [f'./data/{file}' for file in data_files]
if data_files.count('./data/.DS_Store') > 0:
    data_files.remove('./data/.DS_Store')
N_CATEGORIES = len(data_files)

# data_files = [
#     '../data/book.npy',
#     '../data/bus.npy',
#     '../data/camel.npy',
#     '../data/apple.npy', 
#     '../data/airplane.npy',
#     '../data/cactus.npy',
#     '../data/bush.npy',
#     '../data/butterfly.npy',
#     '../data/calculator.npy',
#     '../data/bird.npy',
#     '../data/ambulance.npy',
#     '../data/cake.npy',
#     '../data/beach.npy',
#     '../data/axe.npy',
#     '../data/banana.npy',
#     '../data/calendar.npy',
#     '../data/alarm clock.npy',
#     '../data/basketball.npy',
#     '../data/bed.npy',
#     '../data/barn.npy'
# ]

print(data_files)
print(N_CATEGORIES)

['./data/book.npy', './data/bus.npy', './data/camel.npy', './data/apple.npy', './data/airplane.npy', './data/cactus.npy', './data/bush.npy', './data/butterfly.npy', './data/calculator.npy', './data/bird.npy', './data/ambulance.npy', './data/cake.npy', './data/beach.npy', './data/axe.npy', './data/banana.npy', './data/calendar.npy', './data/alarm clock.npy', './data/basketball.npy', './data/bed.npy', './data/barn.npy']
20


In [5]:
#READING FILES AND POPULATING TOTAL X AND Y
all_drawings = []
all_categories = []

for idx, file in enumerate(data_files):
    data = np.load(file)[:NUM_FILES]
    for d in data:
        reshaped_img = np.array(np.reshape(d, (-1, 28))).astype(np.float32)
        
        all_drawings.append(reshaped_img)
        all_categories.append(idx)

all_drawings = np.array(all_drawings)
all_categories = np.array(all_categories)

### Selecao de dados entre dados de treinamento e teste

Antes de qualquer treinamento ou teste em cima dos dados é necessário separá-los em dados de treinamento e teste sendo 20% dos dados seprados para teste e o restante para treinamento. 

Após essa separação os dados de trinamento serão divididos mais uma vez, sendo que 80% deste dados serao para treinamento do modelo e 20% para validação do treinamento.


In [6]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(all_drawings, all_categories, test_size=0.2, random_state=RANDOM_SEED)

## Estabelecendo uma base
Antes de iniciar a criação da rede convolucional, precisamos de uma base para saber se nosso modelo faz classificaçõeses melhores que modelos aleatórios. Para isso vamos utilizar o classificador Random Forrest para estabelecer um mínimo de performance para nosso modelo convolucional. Dessa forma, se a rede concolucional criada possuir desempenho similar ao classificador aleatório, não podemos dizer que obtivemos qualquer sucesso.

Como o classificador Random forrest recebe dados de treinamento de uma forma diferente (Vetores de no máximo duas dimensões) que uma rede convolucional, precisamos mudar o formato das features para treiná-lo.
Após treinar o classificador Random forest usamos o método `cross_val_score` para separarmos os dados de treinamento em 5 divisoes. Esse método combina 4 divisões para treinamento e a divisão restante para validação. Por fim, obtivemos os seguintes valores de acurácia de validação:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf_drawings = np.array([all_drawings[i:i + RF_NUM_FILES] for i in range(0, len(all_drawings), NUM_FILES)])
rf_drawings = np.concatenate([draw for draw in rf_drawings])

rf_categories = np.array([all_categories[i:i + RF_NUM_FILES] for i in range(0, len(all_categories), NUM_FILES)])
rf_categories = np.concatenate([cat for cat in rf_categories])

X_rf_train_full, X_rf_test, y_rf_train_full, y_rf_test = train_test_split(rf_drawings, rf_categories, test_size=0.2, random_state=RANDOM_SEED)

random_forest_X_train = np.array([x.flatten() for x in X_rf_train_full])
random_forrest_clf = RandomForestClassifier(random_state=RANDOM_SEED, n_jobs=-1)

cv_rf_score = cross_val_score(random_forrest_clf, random_forest_X_train, y_rf_train_full, cv=5, n_jobs=-1)
for idx, val in enumerate(cv_rf_score):
    print(f"{idx+1}:  {round(val*100, 2)}%")

### Montagem da rede convolucional

O modelo criado possui 3 conjuntos de camadas convolucionais seguidas de um max pooling. Os filtros da camada convolucional começam em 8 e vão dobrando, já para o kernel initializer foi utilizado o padrão que é o `glorot_uniform` de tamanho 3x3. Essas primeiras camadas tem o objetivo de destacar as features únicas(detalhes) de cada categoria. Em seguida é realizado o Flatten para transformar todas as camadas em uma array 1D e um Batch Normalization para padronizar os valores, ou seja, centralizar sua média em 0 e o desvio padrão em 1. Também foi adicionada uma camada de Dropout para evitar o overfitting. Na sequência, os inputs normalizados  são passados para uma camada Densa, a qual recebe dois parametros, número de neurônios e função de ativação que sera usada, assim a camada densa recebe as features, multiplica pelos respectivos pesos e soma esta multiplicação para finalmente passar pela funcao de ativação. Em seguida é aplicada mais uma batch normalization e um dropout. Por fim, é aplicada mais uma camada Dense com units = número de categorias, com a função de ativação soft max, para obter o resultado da classificação.

#### Entradas e saídas das camadas
```
(28, 28, 1) -> Primeira camada Conv2D -> (28, 28, 8)
(28, 28, 8) -> Primerira camada Maxpooling2D -> (14, 14, 8)
(14, 14, 8) -> Segunda camada Conv2D -> (14, 14, 16)
(14, 14, 16) -> Segunda camada Maxpooling2D -> (7, 7, 16)
(7, 7, 16) -> Terceira camada Conv2D -> (7, 7, 32)
(7, 7, 32) -> Terceira camada Maxpooling2D -> (4, 4, 32)
(4, 4, 32) -> Flatten -> (512)
(512) -> Dense -> (30)
(30) -> Dense -> (N)
```

In [1]:
slice_index = int(len(X_train_full)*0.8)

X_train = X_train_full[:slice_index][..., np.newaxis]
X_valid = X_train_full[slice_index:][..., np.newaxis]

y_train = y_train_full[:slice_index][..., np.newaxis]
y_valid = y_train_full[slice_index:][..., np.newaxis]

X_test = X_test[..., np.newaxis]

NameError: name 'X_train_full' is not defined

In [11]:
model = keras.models.Sequential([
    Conv2D(filters=8, kernel_size=3,padding="same", activation="relu", input_shape=(IMG_HEIGHT,IMG_WIDTH,1)),
    MaxPool2D(pool_size=2),

    Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(pool_size=2),

    Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(pool_size=2),
    
    Flatten(),
    
    BatchNormalization(),
    Dropout(0.5, seed=RANDOM_SEED),
    Dense(units=30,activation="relu"),
    
    BatchNormalization(),
    Dropout(0.5, seed=RANDOM_SEED),
    Dense(units=N_CATEGORIES, activation="softmax"),
])

In [12]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,epochs = N_EPOCHS, validation_data=(X_valid, y_valid), batch_size=264)

In [ ]:
model.save('./trained_models/')

### Análise do treinamento
Pode-se perceber pelos gráficos 1 e 2 que o modelo apresenta uma melhora na acurácia e no erro a cada iteração. Além disso, a validação acompanha essa melhora do treinamento, ou seja, o modelo apresenta bons resultados para o que ele está sendo treinado e também para dados genéricos(dados que nao estavam presente no treinamento), eliminando a possibilidade de overfitting. Por fim, a acurácia de validação do treinamento e o erro dessa validação são melhores que o próprio treinamento, visto que há duas camadas de dropout durante o treinamento, que eliminam aleatóriamente alguns dados de entrada, mas mantendo a soma total das entradas. Dessa forma, o modelo não se acomoda com os dados de treinamento e é obrigado a ser capaz de generalizar (evitando overfitting).

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(N_EPOCHS)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Treinamento')
plt.plot(epochs_range, val_acc, label='Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treinamento e Validação (1)')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Treinamento')
plt.plot(epochs_range, val_loss, label='Validação')
plt.legend(loc='upper right')
plt.title('Erro de Treinamento e Validação (2)')
plt.show()

In [ ]:
# score = model.evaluate(X_test, y_test)

In [ ]:
model = tf.keras.models.load_model(f'../model/trained_models')

In [ ]:
predictions = model.predict_classes(X_test)

categories = [x.split('/')[-1].split('.')[0] for x in data_files]

### Conclusão

A partir da matriz de confusão gerada com as predições dos dados de teste, pode-se perceber que a média de acurácia é aproximadamente 90%, contudo há algumas classes que se distanciam desse valor. Por exemplo, a classe `bus` acurácia mais baixa que a média e pode-se perceber que há muitos falsos positivos, ou seja, o modelo acha que é um ônibus, mas na verdade não é, e a maioria desses falsos positivos são ambulâncias, o que é bem plausível. Esse mesmo fenômeno ocorre ao contrário. Além disso, isso ocorre com pássaros e aviões e também calendários e calculadoras.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, predictions, normalize='pred')
cmd = ConfusionMatrixDisplay(cm, display_labels=categories)
fig, ax = plt.subplots(figsize=(15,15))
cmd.plot(ax=ax, xticks_rotation='vertical', values_format = '.2f')

Por fim, obtivemos os dados de precision, recall e f1 score. A acurácia final do modelo foi de `88%`. Com esses dados, podemos ver que a classe `beach` possui um precision bem menor que o recall, o que pode ser comprovado na matriz de confusão. O eixo vertical dessa classe(tirando a predição certa) representa os falsos positivos, já o horizontal os falsos negativos. Como o precision é menor a soma desses valores do eixo vertical é maior do que as do eixo horizontal. Além disso, esses valores que representam os falsos positivos estão muito espalhados, o modelo acha que muitas categorias são `beach`, quando não são.

In [ ]:
print(classification_report(y_test, predictions, target_names = categories)) 